<a href="https://colab.research.google.com/github/schadmichael/ParsingQuotes/blob/main/pricing_cap_floor_swaption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#!pip install QuantLib
import QuantLib as ql

# 1. Initialisation des paramètres de base de QuantLib
# Aujourd'hui
today = ql.Date(15, ql.July, 2024)
ql.Settings.instance().evaluationDate = today

# Calendrier, conventions de jour, fréquence
calendar = ql.TARGET()
day_count = ql.Actual360()
settlement_days = 2
settlement_date = calendar.advance(today, ql.Period(settlement_days, ql.Days))

# Courbe des taux (exemple simple: taux plat)
risk_free_rate = 0.03
flat_rate = ql.QuoteHandle(ql.SimpleQuote(risk_free_rate))
discount_curve = ql.YieldTermStructureHandle(
    ql.FlatForward(settlement_date, flat_rate, day_count)
)

print(f"Date d'évaluation: {today}")
print(f"Date de règlement: {settlement_date}")
print(f"Taux sans risque (plat): {risk_free_rate*100}%")

Date d'évaluation: July 15th, 2024
Date de règlement: July 17th, 2024
Taux sans risque (plat): 3.0%


### Évaluation d'un Cap

Un cap est une option qui met un plafond (cap) sur le taux d'intérêt variable payé sur un montant notionnel. Si le taux variable dépasse le taux du cap, le vendeur du cap paie la différence à l'acheteur. Il s'agit en fait d'une série d'options call sur le taux flottant.

In [22]:
# 2. Paramètres du Cap
notional = 1_000_000
cap_rate = 0.035 # Taux de cap (strike)
term = ql.Period(5, ql.Years)
fixed_frequency = ql.Annual # This is a QL.Frequency enum
floating_frequency = ql.Semiannual # This is a QL.Frequency enum

# Schedule des paiements (dates de début et fin pour chaque jambe)
cap_start_date = settlement_date
cap_end_date = calendar.advance(cap_start_date, term)

cap_schedule = ql.Schedule(
    cap_start_date,
    cap_end_date,
    ql.Period(6, ql.Months), # Corrected: use Period with number and TimeUnit for semiannual
    calendar,
    ql.ModifiedFollowing,
    ql.ModifiedFollowing,
    ql.DateGeneration.Forward,
    False
)

# Relinkable handle for the forecasting term structure (ibor_index uses this)
forecasting_term_structure_handle = ql.RelinkableYieldTermStructureHandle()
forecasting_term_structure_handle.linkTo(discount_curve.currentLink()) # Link to our flat discount curve

# Use a standard IborIndex (e.g., Euribor6M) for better compatibility
ibor_index = ql.Euribor6M(forecasting_term_structure_handle)

# Volatilité pour le pricing (exemple simple: volatilité constante)
cap_vol = 0.20 # 20%
cap_vol_handle = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(settlement_date, calendar, cap_vol, day_count)
)

# Création du Cap
cap = ql.CapFloor(
    ql.CapFloor.Cap,
    cap_schedule,
    ibor_index,
    day_count,
    [cap_rate] # Taux de strike doit être une liste
)

# Moteur de pricing (Black Scholes)
cap_engine = ql.BlackCapFloorEngine(discount_curve, cap_vol_handle)
cap.setPricingEngine(cap_engine)

# Prix du Cap
cap_npv = cap.NPV()
print(f"\n--- Cap Pricing ---")
print(f"Taux de Cap: {cap_rate*100}%")
print(f"Volatilité du Cap: {cap_vol*100}%")
print(f"NPV du Cap: {cap_npv:.4f} EUR")

AttributeError: No constructor defined

### Évaluation d'un Floor

Un floor est l'inverse d'un cap. Il fixe un taux d'intérêt minimum (floor) pour les paiements variables. Si le taux variable tombe en dessous du taux du floor, le vendeur du floor paie la différence à l'acheteur. Il s'agit d'une série d'options put sur le taux flottant.

In [19]:
# 3. Paramètres du Floor (similaires au Cap)
floor_rate = 0.025 # Taux de floor (strike)
floor_vol = 0.20 # Même volatilité pour l'exemple
floor_vol_handle = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(settlement_date, calendar, floor_vol, day_count)
)

# Relinkable handle for the forecasting term structure (ibor_index uses this)
forecasting_term_structure_handle = ql.RelinkableYieldTermStructureHandle()
forecasting_term_structure_handle.linkTo(discount_curve.currentLink()) # Link to our flat discount curve

# Use a standard IborIndex (e.g., Euribor6M) for better compatibility
ibor_index = ql.Euribor6M(forecasting_term_structure_handle)

# Création du Floor
floor = ql.CapFloor(
    ql.CapFloor.Floor,
    cap_schedule, # Utilise le même schedule que le cap pour l'exemple
    ibor_index,
    day_count,
    [floor_rate] # Taux de strike doit être une liste
)

# Moteur de pricing
floor_engine = ql.BlackCapFloorEngine(discount_curve, floor_vol_handle)
floor.setPricingEngine(floor_engine)

# Prix du Floor
floor_npv = floor.NPV()
print(f"\n--- Floor Pricing ---")
print(f"Taux de Floor: {floor_rate*100}%")
print(f"Volatilité du Floor: {floor_vol*100}%")
print(f"NPV du Floor: {floor_npv:.4f} EUR")

AttributeError: No constructor defined

### Évaluation d'un Swaption

Un swaption est une option qui donne le droit (mais non l'obligation) d'entrer dans un contrat de swap de taux d'intérêt à une date future spécifiée et à un taux d'intérêt prédéterminé (le taux strike du swap). On peut avoir un swaption payer (droit de payer le fixe) ou un swaption receiver (droit de recevoir le fixe).

In [25]:
# 4. Paramètres du Swaption
# Option d'un an sur un swap de 5 ans
Maturity=1
option_maturity = ql.Period(Maturity, ql.Years)
Tenor=5
swap_term = ql.Period(Tenor, ql.Years)
swaption_strike_rate = 3.2/100 # Taux fixe du swap sous-jacent si l'option est exercée

# Date d'exercice du swaption
exercise_date = calendar.advance(settlement_date, option_maturity)

# Le swap sous-jacent
# (Démarrer à la date d'exercice du swaption et durer 5 ans)
swap_start_date = exercise_date
swap_end_date = calendar.advance(swap_start_date, swap_term)

# Schedule des paiements pour la jambe fixe et flottante du swap
fixed_schedule = ql.Schedule(
    swap_start_date,
    swap_end_date,
    ql.Period(1, ql.Years), # Corrected: use Period with number and TimeUnit for annual
    calendar,
    ql.ModifiedFollowing,
    ql.ModifiedFollowing,
    ql.DateGeneration.Forward,
    False
)

floating_schedule = ql.Schedule(
    swap_start_date,
    swap_end_date,
    ql.Period(6, ql.Months), # Corrected: use Period with number and TimeUnit for semiannual
    calendar,
    ql.ModifiedFollowing,
    ql.ModifiedFollowing,
    ql.DateGeneration.Forward,
    False
)

# Création du swap sous-jacent (payer = payer le fixe, recevoir le flottant)
swap = ql.VanillaSwap(
    ql.VanillaSwap.Payer, # Type de swap: payer le fixe
    notional,
    fixed_schedule,
    swaption_strike_rate, # Taux fixe du swap
    day_count,
    floating_schedule,
    ibor_index,
    0.0, # Ajout du spread (mis à zéro pour l'exemple)
    day_count
)

# Exercice du swaption (européen pour l'exemple)
exercise = ql.EuropeanExercise(exercise_date)

# Création du Swaption
swaption = ql.Swaption(swap, exercise)

# Volatilité du swaption (volatilité de swap, pas de cap/floor)
swaption_vol = 0.25 # 25%
# Corrected: use SwaptionVolatilityStructureHandle for BlackSwaptionEngine
swaption_vol_handle = ql.SwaptionVolatilityStructureHandle(
    ql.ConstantSwaptionVolatility(settlement_date, calendar, ql.Following, swaption_vol, day_count)
)

# Moteur de pricing (Black Scholes pour swaption)
swaption_engine = ql.BlackSwaptionEngine(discount_curve, swaption_vol_handle)
swaption.setPricingEngine(swaption_engine)

# Prix du Swaption
swaption_npv = swaption.NPV()
print(f"\n--- Swaption Pricing ---")
print(f"Maturité de l'option: {option_maturity}")
print(f"Maturité du swap sous-jacent: {swap_term}")
print(f"Taux strike du swap: {swaption_strike_rate*100}%")
print(f"Volatilité du Swaption: {swaption_vol*100}%")
print(f"NPV du Swaption: {swaption_npv:.4f} EUR")


--- Swaption Pricing ---
Maturité de l'option: 1Y
Maturité du swap sous-jacent: 5Y
Taux strike du swap: 3.2%
Volatilité du Swaption: 25.0%
NPV du Swaption: 10869.6649 EUR


In [24]:
exercise_date

Date(17,7,2025)